In [45]:
import os 
import pandas as pd

import plotly.express as px
from IPython.display import HTML

pd.set_option('display.max_columns', None)

ROOTDIR = os.path.dirname(os.getcwd())
DATADIR = os.path.join(ROOTDIR, 'data')

In [47]:
df = pd.read_parquet(os.path.join(DATADIR, 'clean.parquet'))
df.tail()

,date,desc,cat,amount,name,note,ym
2430,2020-08-24 13:14:01,None,None,-5.00,holidays pot,None,2020-08-01
2431,2020-08-24 13:15:29,None,None,-5.00,large expenses pot,None,2020-08-01
2432,2020-08-24 13:29:01,monzo plus,general,-5.00,None,None,2020-08-01
2433,2020-08-24 15:54:56,sent from monzo,health,-21.50,tf fitness centre,sent from monzo,2020-08-01
2434,2020-08-24 17:51:56,tesco stores 6181 berwick gbr,groceries,-20.26,tesco,None,2020-08-01


In [42]:
def load_data():
    url = 'https://docs.google.com/spreadsheets/d/'
    sheet_id = '1N9obnCF3fdjq2sOL0uvkVhbhpmZFOtkssXjCdTGfJak'
    tab_id = '1646235209'
    path = f'{url}{sheet_id}/export?format=csv&gid={tab_id}'
    return pd.read_csv(path, parse_dates={'date': ['Date', 'Time']})

df = load_data()
df.head()

,date,Transaction ID,Type,Name,Emoji,Category,Amount,Currency,Local amount,Local currency,Notes and #tags,Address,Receipt,Description,Category split
0,2018-11-11 11:45:52,tx_00009ceGW7oBS9iQEHznxB,Faster payment,GUNZINGER,NaN,General,200.00,GBP,200.00,GBP,FGtoFG,NaN,NaN,FGtoFG,NaN
1,2018-11-11 11:52:40,tx_00009ceH7gJC93F2Z8eJ7Z,Faster payment,Fabian Gunzinger,NaN,General,-200.00,GBP,-200.00,GBP,P9811069,NaN,NaN,P9811069,NaN
2,2018-11-18 16:19:23,tx_00009ctAWUp6CRG1sMr9uL,Card payment,Waitrose & Partners,🍏,Groceries,-2.99,GBP,-2.99,GBP,NaN,116-128 North End Road,NaN,WAITROSE LONDON GBR,NaN
3,2018-11-18 16:21:02,tx_00009ctAfeYPOFHTYTDFab,Card payment,Tesco,🍏,Groceries,-5.05,GBP,-5.05,GBP,NaN,112 North End Road,NaN,TESCO STORES 5103 HAMMERSMITH GBR,NaN
4,2018-11-19 05:35:02,tx_00009cuJX410Rx7cf2Wo0v,Card payment,Transport for London,🚇,Transport,-12.40,GBP,-12.40,GBP,"Travel charge for Monday, 19 Nov",NaN,NaN,TfL Travel Charge TFL.gov.uk/CP GBR,NaN


# Resources

- Using Plotly for Jupyter Lab and Fastpages ([link](https://colab.research.google.com/github/binnisb/blog/blob/master/_notebooks/2020-04-02-Plotly-in-lab.ipynb#scrollTo=TWW5ogvQPEk1))